<a href="https://colab.research.google.com/github/Shruthi-Arun/Macroeconomic-Indicators-for-Stock-Dynamics/blob/main/Notebooks/ESE5380_Data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1 - Data Collection and Pre-Processing
A combined macroeconomic–financial dataset is created using indicators from the Federal Reserve Economic Database (FRED) and stock market data from Yahoo Finance. <br><br>

Eight key U.S. macroeconomic indicators are retrieved through the fredapi package: inflation (CPIAUCSL), unemployment (UNRATE), interest rate (FEDFUNDS), real GDP (GDPC1), industrial output (INDPRO), money supply (M2SL), consumer confidence (UMCSENT), and 10-year treasury yield (GS10). <br><br>

S&P 500 index data (^GSPC) is downloaded using yfinance, and monthly percentage returns are calculated to represent market performance.
All series are standardized to a monthly frequency and aligned to month-end timestamps for uniformity. <br><br>
Quarterly indicators such as GDP are forward-filled to fill missing months, and minor residual gaps across indicators are handled through forward and backward filling.
<br><br>
The final dataset spans 1955–2025 with 849 monthly observations across nine variables.

In [ ]:
!pip install fredapi

In [ ]:
# --- Import libraries ---
from fredapi import Fred
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- Initialize FRED ---
fred = Fred(api_key='70b316692bf4c602d54fe9c4c752a18c')

In [ ]:
# --- FRED Series IDs ---
fred_series = {
    'Inflation': 'CPIAUCSL',
    'Unemployment': 'UNRATE',
    'Interest_Rate': 'FEDFUNDS',
    'GDP': 'GDPC1',
    'Industrial_Output': 'INDPRO',
    'Money_Supply': 'M2SL',
    'Consumer_Confidence': 'UMCSENT',
    'Treasury_Yield': 'GS10'
}

In [ ]:
# --- FRED (pull raw, then make monthly, forward-fill slow series) ---
df_fred = pd.DataFrame({k: fred.get_series(v) for k, v in fred_series.items()})
df_fred.index = pd.to_datetime(df_fred.index)

In [ ]:
# monthly month-end index, forward-fill for quarterly/irregular
df_fred = df_fred.resample('M').ffill()
df_fred[['GDP','Consumer_Confidence']] = df_fred[['GDP','Consumer_Confidence']].ffill()

/tmp/ipython-input-1362262850.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_fred = df_fred.resample('M').ffill()


In [ ]:
# align to month-end timestamps
df_fred.index = df_fred.index.to_period('M').to_timestamp('M')

In [ ]:
# --- S&P 500 (pull MAX daily, then resample to month-end yourself) ---
hist = yf.Ticker("^GSPC").history(period="max", auto_adjust=True)
hist = hist[['Close']].rename(columns={'Close':'SP500_Close'})
hist.index = pd.to_datetime(hist.index)

In [ ]:
# month-end prices & returns
sp500_m = hist.resample('M').last()
sp500_ret = sp500_m['SP500_Close'].pct_change().dropna() * 100
sp500_ret.name = 'SP500_Returns'
sp500_ret.index = sp500_ret.index.to_period('M').to_timestamp('M')

/tmp/ipython-input-1867777784.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  sp500_m = hist.resample('M').last()
/tmp/ipython-input-1867777784.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  sp500_ret.index = sp500_ret.index.to_period('M').to_timestamp('M')


In [ ]:
# --- Merge on intersection only (clean econometrics) ---
df = pd.merge(df_fred, sp500_ret, left_index=True, right_index=True, how='inner')

In [ ]:
# restrict to a broad era
df = df.loc['1955':'2025-10']  # earliest limited by GS10(1962), M2(1959), FEDFUNDS(1954), SP500(1950)

In [ ]:
print("Final merged dataset:", df.shape)
print("Range:", df.index.min().strftime("%Y-%m"), "→", df.index.max().strftime("%Y-%m"))
print("\nSeries coverage:")
for col in df.columns:
    s = df[col].dropna()
    print(f"{col:<22} {s.index.min().strftime('%Y-%m')} → {s.index.max().strftime('%Y-%m')} (n={len(s)})")

Final merged dataset: (850, 9)
Range: 1955-01 → 2025-10

Series coverage:
Inflation              1955-01 → 2025-09 (n=849)
Unemployment           1955-01 → 2025-08 (n=848)
Interest_Rate          1955-01 → 2025-10 (n=850)
GDP                    1955-01 → 2025-10 (n=850)
Industrial_Output      1955-01 → 2025-08 (n=848)
Money_Supply           1959-01 → 2025-09 (n=801)
Consumer_Confidence    1955-01 → 2025-10 (n=850)
Treasury_Yield         1955-01 → 2025-10 (n=850)
SP500_Returns          1955-01 → 2025-10 (n=850)


In [ ]:
df.head()

,Inflation,Unemployment,Interest_Rate,GDP,Industrial_Output,Money_Supply,Consumer_Confidence,Treasury_Yield,SP500_Returns
1955-01-31,26.77,4.9,1.39,3020.746,19.2735,NaN,87.0,2.61,1.806563
1955-02-28,26.82,4.7,1.29,3020.746,19.5154,NaN,95.9,2.65,0.354893
1955-03-31,26.79,4.6,1.35,3020.746,19.9724,NaN,95.9,2.68,-0.489653
1955-04-30,26.79,4.7,1.43,3069.910,20.2143,NaN,95.9,2.75,3.772546
1955-05-31,26.77,4.3,1.43,3069.910,20.5369,NaN,99.1,2.76,-0.131716


In [ ]:
df.to_csv('macro_stock_merged_1955_2025.csv')
print("\n💾 Saved macro_stock_merged_1955_2025.csv")


💾 Saved macro_stock_merged_1955_2025.csv


In [ ]:
# converting Unnamed0 to date
df = pd.read_csv('macro_stock_merged_1955_2025.csv')
print(df.columns)

df = df.rename(columns={'Unnamed: 0': 'Date'})

# Convert to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

df.info()


Index(['Unnamed: 0', 'Inflation', 'Unemployment', 'Interest_Rate', 'GDP',
       'Industrial_Output', 'Money_Supply', 'Consumer_Confidence',
       'Treasury_Yield', 'SP500_Returns'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 849 entries, 1955-01-31 to 2025-09-30
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Inflation            849 non-null    float64
 1   Unemployment         848 non-null    float64
 2   Interest_Rate        849 non-null    float64
 3   GDP                  849 non-null    float64
 4   Industrial_Output    848 non-null    float64
 5   Money_Supply         801 non-null    float64
 6   Consumer_Confidence  849 non-null    float64
 7   Treasury_Yield       849 non-null    float64
 8   SP500_Returns        849 non-null    float64
dtypes: float64(9)
memory usage: 66.3 KB


In [ ]:
df.head()

,Inflation,Unemployment,Interest_Rate,GDP,Industrial_Output,Money_Supply,Consumer_Confidence,Treasury_Yield,SP500_Returns
Date,,,,,,,,,
1955-01-31,26.77,4.9,1.39,3020.746,19.2735,NaN,87.0,2.61,1.806563
1955-02-28,26.82,4.7,1.29,3020.746,19.5154,NaN,95.9,2.65,0.354893
1955-03-31,26.79,4.6,1.35,3020.746,19.9724,NaN,95.9,2.68,-0.489653
1955-04-30,26.79,4.7,1.43,3069.910,20.2143,NaN,95.9,2.75,3.772546
1955-05-31,26.77,4.3,1.43,3069.910,20.5369,NaN,99.1,2.76,-0.131716


In [ ]:
df = pd.read_csv("macro_stock_merged_1955_2025.csv", parse_dates=["Unnamed: 0"], index_col="Unnamed: 0")

# Rename index column
df.index.name = "Date"

# Quick overview
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 849 entries, 1955-01-31 to 2025-09-30
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Inflation            849 non-null    float64
 1   Unemployment         848 non-null    float64
 2   Interest_Rate        849 non-null    float64
 3   GDP                  849 non-null    float64
 4   Industrial_Output    848 non-null    float64
 5   Money_Supply         801 non-null    float64
 6   Consumer_Confidence  849 non-null    float64
 7   Treasury_Yield       849 non-null    float64
 8   SP500_Returns        849 non-null    float64
dtypes: float64(9)
memory usage: 66.3+ KB
None
            Inflation  Unemployment  Interest_Rate       GDP  \
Date                                                           
1955-01-31      26.77           4.9           1.39  3020.746   
1955-02-28      26.82           4.7           1.29  3020.746   
1955-03-31      26.79           

In [ ]:
df.index = pd.to_datetime(df.index, errors='coerce')


In [ ]:
print(type(df.index))


<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


In [ ]:
df = df.dropna(subset=['Money_Supply'])
print(df.isna().sum())

# Optional: check the new start date
# print("New start date after dropping NaNs:", df.index.min())

Inflation              0
Unemployment           1
Interest_Rate          0
GDP                    0
Industrial_Output      1
Money_Supply           0
Consumer_Confidence    0
Treasury_Yield         0
SP500_Returns          0
dtype: int64
New start date after dropping NaNs: 1959-01-31 00:00:00


In [ ]:
df['Unemployment'] = df['Unemployment'].ffill()
df['Industrial_Output'] = df['Industrial_Output'].ffill()

In [ ]:
df.isna().sum()

,0
Inflation,0
Unemployment,0
Interest_Rate,0
GDP,0
Industrial_Output,0
Money_Supply,0
Consumer_Confidence,0
Treasury_Yield,0
SP500_Returns,0


In [ ]:
df.to_csv('macro_data_cleaned.csv', index=True)